In [1]:
import os
import uuid
import pickle
import boto3
import pandas as pd

import mlflow

from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import make_pipeline

In [2]:
import dotenv

In [3]:
dotenv.load_dotenv(".env")

True

In [4]:
year = 2021
month = 2
taxi_type = 'green'

# input_file = f'https://s3.amazonaws.com/nyc-tlc/trip+data/{taxi_type}_tripdata_{year:04d}-{month:02d}.parquet'
input_file =f'https://d37ci6vzurychx.cloudfront.net/trip-data/{taxi_type}_tripdata_{year:04d}-{month:02d}.parquet'
output_dir = f"output/{taxi_type}"
os.makedirs(output_dir,exist_ok=True)
output_file = f'{output_dir}/{year:04d}-{month:02d}.parquet'

RUN_ID = os.getenv('RUN_ID', 'cc36795ca2fd48e8a176194f450c0ade')

In [5]:
def upload_file_to_s3(file_name,bucket_name,object_key):
    s3 = boto3.client('s3')
    s3.upload_file(file_name, bucket_name, object_key)

In [6]:
# def generate_uuids(n):
#     ride_ids = []
#     for i in range(n):
#         ride_ids.append(str(uuid.uuid4()))
#     return ride_ids

def read_dataframe(filename: str):
    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.dt.total_seconds() / 60
    df = df[(df.duration >= 1) & (df.duration <= 60)]
    
    df['ride_id'] = df.apply(lambda x:str(uuid.uuid4()),axis = 1)

    return df


def prepare_dictionaries(df: pd.DataFrame):
    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']

    categorical = ['PU_DO']
    numerical = ['trip_distance']
    dicts = df[categorical + numerical].to_dict(orient='records')
    return dicts

In [7]:
def load_model(run_id):
    logged_model  = f"s3://s3-mlflow-artifacts-storage/mlflow/12/{RUN_ID}/artifacts/model" 
    model = mlflow.pyfunc.load_model(logged_model)
    return model


def apply_model(input_file, run_id, output_file):

    df = read_dataframe(input_file)
    dicts = prepare_dictionaries(df)

    
    model = load_model(run_id)
    y_pred = model.predict(dicts)

    df_result = pd.DataFrame()
    df_result['ride_id'] = df['ride_id']
    df_result['lpep_pickup_datetime'] = df['lpep_pickup_datetime']
    df_result['PULocationID'] = df['PULocationID']
    df_result['DOLocationID'] = df['DOLocationID']
    df_result['actual_duration'] = df['duration']
    df_result['predicted_duration'] = y_pred
    df_result['diff'] = df_result['actual_duration'] - df_result['predicted_duration']
    df_result['model_version'] = run_id
    
    df_result.to_parquet(output_file, index=False)
    return df_result

In [8]:
df = apply_model(input_file=input_file, run_id=RUN_ID, output_file=output_file)

2023/06/18 10:28:32 WARNING mlflow.pyfunc: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - mlflow (current: 2.3.2, required: mlflow==2.3)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.


In [9]:
df.head()

,ride_id,lpep_pickup_datetime,PULocationID,DOLocationID,actual_duration,predicted_duration,diff,model_version
0,011bd633-3c73-47a4-8bd2-490f3100dd2b,2021-02-01 00:34:03,130,205,17.916667,18.168946,-0.252279,cc36795ca2fd48e8a176194f450c0ade
1,f57da1d7-1883-4427-bd6a-129f32da062b,2021-02-01 00:04:00,152,244,6.500000,7.290331,-0.790331,cc36795ca2fd48e8a176194f450c0ade
2,cd475665-8241-4c87-845a-31062c27d3fd,2021-02-01 00:18:51,152,48,15.250000,21.278875,-6.028875,cc36795ca2fd48e8a176194f450c0ade
3,2a97c274-5915-4783-ae34-96556173b281,2021-02-01 00:53:27,152,241,18.233333,25.008761,-6.775428,cc36795ca2fd48e8a176194f450c0ade
4,481866bd-71fb-4525-991d-d56732a13f19,2021-02-01 00:57:46,75,42,8.966667,9.380744,-0.414077,cc36795ca2fd48e8a176194f450c0ade


In [14]:
object_key = f"taxi_type={taxi_type}/year={year:04d}/month={month:02d}/{RUN_ID}.parquet"

In [15]:
upload_file_to_s3(output_file,'nyc-duration-prediction',object_key)

In [10]:
!ls output/green/

2021-02.parquet
